In [0]:
from tensorflow.python.framework import graph_util
import tensorflow as tf
# TensorFlow Graph visualizer code
import numpy as np
from IPython.display import clear_output, Image, display, HTML


In [0]:

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
     """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
   
      
    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [25]:
with tf.Graph().as_default() as graph:
  with tf.Session() as sess:

    a = tf.placeholder(dtype=tf.float32, name='a')
    b = tf.placeholder(dtype=tf.float32, name='b')
    adder_node = tf.add(a,b)
    add_and_mul = adder_node * 2.
    y = tf.nn.sigmoid(add_and_mul, name='output')
    
    print(sess.run(add_and_mul, {a:3., b:4.}))
    graph_def = graph_util.convert_variables_to_constants(
      sess, graph.as_graph_def(),['output'])
    show_graph(graph_def)

14.0
INFO:tensorflow:Froze 0 variables.


INFO:tensorflow:Froze 0 variables.


Converted 0 variables to const ops.
